# Multiple Sheets

In [75]:
import pandas as pd

In [76]:
df_1 = pd.read_excel('data/shifts.xlsx', sheet_name='Sheet1')
df_2 = pd.read_excel('data/shifts.xlsx', sheet_name='Sheet2')
df_3 = pd.read_excel('data/shift_3.xlsx')

In [77]:
df_all = pd.concat([df_1, df_2, df_3], sort=False)
df_all

,Shift,Region,Sale Rep,Product,Cost Per,Units Sold
0,1,South,Deborah,Binder,30,55
1,1,East,Alice,Stapler,25,113
2,1,North,Ellis,Folder,17,97
3,1,North,Ellis,Pencil,5,83
4,1,South,Alice,Binder,30,55
...,...,...,...,...,...,...
94,3,East,Ellis,Binder,30,177
95,3,North,Bob,Pencil,5,54
96,3,South,Charlie,Pen,10,196
97,3,North,Charlie,Paper,15,80


In [78]:
df_all.loc[50]

,Shift,Region,Sale Rep,Product,Cost Per,Units Sold
50,1,North,Deborah,Pencil,5,87
50,2,East,Deborah,Binder,30,160
50,3,North,Bob,Binder,30,79


In [79]:
df_all.groupby(['Shift']).mean()['Units Sold']

Shift
1    121.434343
2    126.060606
3    124.858586
Name: Units Sold, dtype: float64

In [80]:
df_all.to_excel('output/all_shifts.xlsx', index=None)

In [81]:
from openpyxl import load_workbook
from openpyxl.styles import Font

In [82]:
wb = load_workbook('output/all_shifts.xlsx')
ws = wb.active

In [83]:
total_col = ws['G1']
total_col.font = Font(bold=True)
total_col.value = 'Total'

In [84]:
e_col, f_col = ['E', 'F']

for row in range(2, 298):
    result_cell = f'G{row}'
    e_value = ws[e_col + str(row)].value
    f_value = ws[f_col + str(row)].value

    ws[result_cell] = e_value * f_value

In [85]:
wb.save('output/totaled.xlsx')

# Converting Data

In [86]:
from openpyxl.utils.dataframe import dataframe_to_rows

In [87]:
wb = load_workbook('data/regions.xlsx')
ws = wb.active

In [88]:
df = pd.read_excel('output/all_shifts.xlsx')
df1 = df[['Sale Rep', 'Cost Per', 'Units Sold']]

In [89]:
df1['Total'] = df1['Cost Per'] * df1['Units Sold']
df1

C:\Users\jagma\AppData\Local\Temp\ipykernel_8744\1648019548.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Total'] = df1['Cost Per'] * df1['Units Sold']


,Sale Rep,Cost Per,Units Sold,Total
0,Deborah,30,55,1650
1,Alice,25,113,2825
2,Ellis,17,97,1649
3,Ellis,5,83,415
4,Alice,30,55,1650
...,...,...,...,...
292,Ellis,30,177,5310
293,Bob,5,54,270
294,Charlie,10,196,1960
295,Charlie,15,80,1200


In [90]:
rows = dataframe_to_rows(df1, index=False)
rows

<generator object dataframe_to_rows at 0x0000013997960BA0>

In [91]:
for r_idx, row in enumerate(rows, 1):
    for c_idx, col in enumerate(row, 6):
        ws.cell(row=r_idx, column=c_idx, value=col)

In [92]:
wb.save('output/combined.xlsx')

 # Large Spreadsheets

In [93]:
import numpy as np

In [94]:
wb = load_workbook('data/Template.xlsx')
ws = wb.active

In [95]:
df = pd.read_csv(
    'data/BostonCrimeDataset.csv',
    encoding='utf-8',
    dtype={
        'INCIDENT_NUMBER': str,
        'OFFENSE_CODE': str,
        'OFFENSE_CODE_GROUP': str,
        'OFFENSE_DESCRIPTION': str,
        'DISTRICT': str,
        'REPORTING_AREA': str,
        'SHOOTING': str,
        'YEAR': str,
        'MONTH': str,
        'DAY_OF_WEEK': str,
        'HOUR': str
    }
)

In [96]:
df1 = df[df['OFFENSE_CODE_GROUP'] == 'Counterfeiting']
df1 = df1.replace(np.nan, 'N/A', regex=True)

In [97]:
total_crimes = len(df.index)
counterfeit = len(df1.index)

perc_crimes = round((counterfeit / total_crimes) * 100, 2)

In [98]:
ws['O8'].value = total_crimes
ws['P8'].value = counterfeit
ws['Q8'].value = perc_crimes

In [99]:
df1['Count'] = 1

df2 = df1.groupby(['DISTRICT', 'YEAR']).count()['Count'].unstack(level=0)
df2.drop(columns='N/A', inplace=True)

In [100]:
rows = dataframe_to_rows(df2)

for r_idx, row in enumerate(rows, 8):
    for c_idx, value in enumerate(row, 1):
        ws.cell(row=r_idx, column=c_idx, value=value)

In [101]:
wb.save('output/crime_report.xlsx')

# Advanced Graphing

In [102]:
from openpyxl.chart import BarChart, PieChart, Series, Reference

In [107]:
wb = load_workbook('output/crime_report.xlsx')
ws = wb.active

In [108]:
chart = BarChart()
data = Reference(ws, min_row=10, min_col=1, max_row=14, max_col=13)
labels = Reference(ws, min_row=8, min_col=2, max_row=8, max_col=13)

chart.add_data(data, from_rows=True, titles_from_data=True)
chart.set_categories(labels)
chart.title = 'Counterfeit Crimes by Distrinct'

chart.height = 4.56
chart.width = 20.3

In [109]:
chart2 = PieChart()
data = Reference(ws, min_col=15, max_col=16, min_row=8, max_row=8)
labels = Reference(ws, min_col=15, max_col=16, min_row=7, max_row=7)

chart2.add_data(data, from_rows=True)
chart2.title = '%Counterfeit Crimes'

chart2.height = 4.56
chart2.width = 8.45

In [110]:
ws.add_chart(chart, 'B15')
ws.add_chart(chart2, 'N15')
wb.save('output/lines.xlsx')